In [1]:
!pip install split_folders
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import splitfolders
import os

os.makedirs('output')
os.makedirs('output/train')
os.makedirs('output/val')
os.makedirs('output/test')

loc = "plantvillage dataset\color"

splitfolders.ratio(loc,output = "output",seed = 42,ratio = (0.80,.1,.1))

In [ ]:
import os

for dirpath,dirname,filename in os.walk("./output"):
  print(f"There are {len(dirname)} directories and {len(filename)} images in '{dirpath}'.")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dir = "./output/train"
val_dir = "./output/val"
test_dir  = "./output/test"


train_data = image_dataset_from_directory(train_dir,label_mode="categorical",image_size = (224,224),
                                          batch_size = 32,seed = 42,shuffle = True)


val_data = image_dataset_from_directory(val_dir,label_mode="categorical",image_size = (224,224),
                                          batch_size = 32,seed = 42,shuffle = False)


test_data = image_dataset_from_directory(test_dir,label_mode="categorical",image_size = (224,224),
                                          batch_size = 32,seed = 42,shuffle = False)

In [ ]:
base_model = tf.keras.applications.DenseNet169(include_top = False)
base_model.trainable = False

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
data_aug = tf.keras.Sequential([
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomFlip("horizontal"),
    preprocessing.Rescaling(1/255.)
],name = "data_augmentation_layer")

In [ ]:
inputs = layers.Input(shape = (224,224,3),name = "Input_layer")
x = data_aug(inputs)
x = base_model(x)
x = layers.GlobalAveragePooling2D(name = "Global_average_pooling_layer")(x)
x = layers.Dense(32,activation = "relu",kernel_initializer = tf.keras.initializers.HeNormal(),name = "Dense_layer_1")(x)
x = layers.Dense(38,name = "Dense_layer_2")(x)
outputs = layers.Activation("softmax",dtype = tf.float32)(x)
model_1 = tf.keras.Model(inputs,outputs)

In [ ]:
history_1 = model_1.fit(train_data,epochs = 1,steps_per_epoch=len(train_data),
                        validation_data = val_data,validation_steps = int(0.25*len(val_data)),
                        callbacks = [early_stop,reduce_lr]) 